In [ ]:
import pandas as pd
import requests
import time
import os

# Configuration
llama_model = "llama3.1:70b"
ollama_url = "http://............./api/generate" #Replace with a valid URL
headers = {"Content-Type": "application/json"}

# Function to call LLaMA 3.1 via API
def call_llama(prompt: str, instructions: str) -> str:
    full_prompt = f"{instructions}\n\n{prompt}"
    payload = {
        "model": llama_model,
        "prompt": full_prompt,
        "stream": False,
        "options": {
            "temperature": 0
        }
    }

    for attempt in range(3):
        try:
            response = requests.post(ollama_url, json=payload, headers=headers)
            response.raise_for_status()
            result = response.json()["response"].strip()

            # Ensure response is a single letter A–D
            for option in ["A", "B", "C", "D"]:
                if result.strip().upper().startswith(option):
                    return option
            return result[:1].upper()  # fallback
        except Exception as e:
            print(f"❌ Error: {e}. Retrying in 5 seconds...")
            time.sleep(5)
    return "Error"

# System instructions for LLaMA
instructions = (
    "You will be shown a sentence with a placeholder [  ] and four options labeled A, B, C, and D.\n"
    "Choose the option that best completes the sentence in a natural and contextually appropriate way.\n"
    "Respond with only one letter: A, B, C, or D."
)

# Main processing loop per file
def process_file_llama(filename):
    df = pd.read_csv(filename)
    output_path = filename.replace(".csv", "_llama_output_new.csv")

    # Resume if partially done
    if os.path.exists(output_path):
        df_out = pd.read_csv(output_path)
        start_index = df_out["LLaMA Choice"].notna().sum()
    else:
        df["LLaMA Choice"] = ""
        df_out = df.copy()
        start_index = 0

    print(f"📝 Starting from row {start_index} for {filename}")

    for i in range(start_index, len(df_out)):
        row = df_out.iloc[i]
        prompt = (
            f"Domain: {row['Domain']}\n"
            f"Prompt: {row['Prompt']}\n"
            f"A: {row['A']}\n"
            f"B: {row['B']}\n"
            f"C: {row['C']}\n"
            f"D: {row['D']}\n"
            "Which option best completes the sentence?"
        )
        choice = call_llama(prompt, instructions)
        df_out.at[i, "LLaMA Choice"] = choice
        df_out.to_csv(output_path, index=False)
        print(f"✅ [{i+1}/{len(df_out)}] Saved choice: {choice}")

    print(f"🎉 Completed and saved: {output_path}")
    return df_out

# Run on all three datasets
for file in ["High.csv", "Mid.csv", "Low.csv"]:
    process_file_llama(file)


📝 Starting from row 0 for High.csv
